<a href="https://colab.research.google.com/github/SarahWohlford/SW_Projects-in-Machine-Learning-and-AI_P/blob/main/Proj_HW5_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is kinda messy im sorry

It was hard

Some code blocks are duplacits with some changes


Task 3 (55 points): NLP and Attention Mechanism
Part 1 (10 points): Implement the scaled dot-product attention as discussed in class
(lecture 14) from scratch (use NumPy and pandas only, no deep learning libraries are
allowed for this step).

In [ ]:
import numpy as np
import pandas as pd

def scaled_dot_product_attention(Q, K, V):

    # Dot product between Q and K^T
    matmul_qk = np.matmul(Q, K.T)  # shape (..., seq_len_q, seq_len_k)

    #  Scale by sqrt(dk)
    dk = K.shape[-1]
    scaled_attention_logits = matmul_qk / np.sqrt(dk)

    #  Apply softmax to get attention weights
    attention_weights = np.exp(scaled_attention_logits)
    attention_weights /= np.sum(attention_weights, axis=-1, keepdims=True)

    # Multiply weights with V
    output = np.matmul(attention_weights, V)

    return output, attention_weights


Part 2 (10 points): Pick any encoder-decoder seq2seq model (as discussed in class) and
integrate the scaled dot-product attention in the encoder architecture. You may come
up with your own technique of integration or adopt one from literature. Hint: See
Bahdanau or Luong attention paper presented in class (lecture 14).

In [1]:
import tensorflow as tf
import numpy as np

def scaled_dot_product_attention(Q, K, V, mask=None):

    matmul_qk = tf.matmul(Q, K, transpose_b=True)                 # (batch, 1, seq_len)
    dk = tf.cast(tf.shape(K)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(dk)                         # scale
    if mask is not None:
        logits += (mask * -1e9)
    weights = tf.nn.softmax(logits, axis=-1)                      # (batch, 1, seq_len)
    output = tf.matmul(weights, V)                                # (batch, 1, d_model)
    return output, weights

class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super().__init__()
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(
            enc_units, return_sequences=True, return_state=True
        )

    def call(self, x, hidden_state):
        x = self.embedding(x)                                       # (batch, seq_len, emb)
        output, h, c = self.lstm(x, initial_state=hidden_state)
        # output: (batch, seq_len, enc_units)
        return output, (h, c)

    def initialize_hidden_state(self, batch_size):
        return (tf.zeros((batch_size, self.enc_units)),
                tf.zeros((batch_size, self.enc_units)))


class DecoderWithAttention(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super().__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(
            dec_units, return_sequences=True, return_state=True
        )
        # We’ll concat context + embedding before feeding to LSTM
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden_state, enc_output, enc_mask=None):
        """
        x: (batch, 1) — current input token
        hidden_state: tuple (h, c) each (batch, dec_units)
        enc_output: (batch, seq_len, dec_units)
        enc_mask: (batch, 1, seq_len) or None
        """
        #  embed the decoder input
        x = self.embedding(x)                                       # (batch, 1, emb)
        # compute attention context using last hidden state as query
        h_prev = tf.expand_dims(hidden_state[0], 1)                 # (batch, 1, dec_units)
        context, attn_weights = scaled_dot_product_attention(
            h_prev, enc_output, enc_output, mask=enc_mask
        )
        # concat context + embed
        x = tf.concat([context, x], axis=-1)                        # (batch, 1, emb+units)
        # pass through LSTM
        output, h, c = self.lstm(x, initial_state=hidden_state)
        # project to vocab
        logits = self.fc(output)                                    # (batch, 1, vocab_size)
        return logits, (h, c), attn_weights




Part 4 (30 points): In this part you are required to implement a simplified Transformer
model from scratch (using Python and NumPy/PyTorch/TensorFlow with minimal high level abstractions) and apply it to a machine translation task (e.g., English-to-French or
English-to-German translation) using the same dataset from part 3


Create a function or layer to generate positional encodings that will be added to the input embeddings to inject sequence order information.


In [ ]:
def positional_encoding(position, d_model):

    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / d_model)

    # apply sin
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return pos_encoding



Build a multi-head attention mechanism using the scaled dot-product attention function as a base. This will involve splitting the input into multiple heads, applying scaled dot-product attention to each head, and then concatenating the results.


In [ ]:
import tensorflow as tf
import numpy as np

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        #Split the last dimension into (num_heads, depth).
        #Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)

        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def combine_heads(self, x, batch_size):
        #Combine the heads back to (batch_size, seq_len_q, d_model)
        x = tf.transpose(x, perm=[0, 2, 1, 3])
        return tf.reshape(x, (batch_size, -1, self.d_model))

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v)

        # Combine the heads back
        concat_attention = self.combine_heads(scaled_attention, batch_size)  # (batch_size, seq_len_q, d_model)

        # Final linear layer
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights


rewrite the scaled_dot_product_attention function using TensorFlow operations so it can be used within the TensorFlow MultiHeadAttention layer.



In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
    """
    Compute scaled dot-product attention using TensorFlow operations.

    Args:
        q: Query tensor of shape (..., seq_len_q, depth)
        k: Key tensor of shape (..., seq_len_k, depth)
        v: Value tensor of shape (..., seq_len_v, depth_v)
        mask: Optional mask tensor (broadcastable to (..., seq_len_q, seq_len_k)). Defaults to None.

    Returns:
        output: Attention output tensor (..., seq_len_q, depth_v)
        attention_weights: Attention weights tensor (..., seq_len_q, seq_len_k)
    """
    #Dot product between Q and K^T
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    #cale by sqrt(dk)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)


    if mask is not None:
        scaled_attention_logits += (mask * -1e9) # Add a large negative number to masked positions

    # Apply softmax to get attention weights
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    #Multiply weights with V
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights


Single encoder layer that includes multi-head self-attention, a feed-forward network, residual connections, and layer normalization as described in the simplified architecture (2 layers total).


In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        # Multi-head attention sublayer
        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        # Feed-forward network sublayer
        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

Build the full encoder by stacking the specified number of encoder layers (2 layers).


In [ ]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, ff_dim, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, ff_dim, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        seq_len = tf.shape(x)[1]

        # Add embedding and positional encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

Decoder layer that includes masked multi-head self-attention (to prevent attending to future tokens), multi-head attention over the encoder's output, a feed-forward network, residual connections, and layer normalization (2 layers total).


In [ ]:
import tensorflow as tf

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)  # Masked self-attention
        self.mha2 = MultiHeadAttention(d_model, num_heads)  # Encoder-decoder attention

        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        # Masked multi-head self-attention
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(x + attn1)

        # Multi-head attention over encoder output
        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # Use padding_mask here
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(out1 + attn2)

        # Feed-forward network
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(out2 + ffn_output)

        return out3, attn_weights_block1, attn_weights_block2

Build the full decoder by stacking the specified number of decoder layers (2 layers).


In [ ]:
import tensorflow as tf
import numpy as np # Keep numpy import for positional_encoding if it's not a tf.keras.layers.Layer

class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, ff_dim, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, ff_dim, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        # Add embedding and positional encoding.
        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights[f'decoder_layer{i+1}_block1'] = block1
            attention_weights[f'decoder_layer{i+1}_block2'] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights


Transformer model that combines the encoder and decoder.


In [ ]:
import tensorflow as tf

class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, ff_dim, input_vocab_size, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, ff_dim,
                               input_vocab_size, maximum_position_encoding, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, ff_dim,
                               target_vocab_size, maximum_position_encoding, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size, activation='softmax')


    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):

        enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

        return final_output, attention_weights


Utilize the data loading and subword tokenization setup from Part 3 (using the ted_hrlr_translate/pt_to_en dataset ) to prepare the data for training.


In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Load the dataset
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

# Create subword tokenizers
tokenizer_pt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)

tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train_examples), target_vocab_size=2**13)

# Print vocabulary sizes
print(f"Portuguese vocabulary size: {tokenizer_pt.vocab_size}")
print(f"English vocabulary size: {tokenizer_en.vocab_size}")

# Define maximum sequence length
MAX_LENGTH = 40

Portuguese vocabulary size: 8214
English vocabulary size: 8087


Apply the encoding and padding function to the dataset, then filter, cache, shuffle, and batch the data.



In [2]:
def encode_and_pad(pt, en):
    pt_tokens = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(pt.numpy()) + [tokenizer_pt.vocab_size + 1]
    en_tokens = [tokenizer_en.vocab_size] + tokenizer_en.encode(en.numpy()) + [tokenizer_en.vocab_size + 1]

    # Pad and truncate
    pt_padded = pt_tokens[:MAX_LENGTH] + [0] * max(0, MAX_LENGTH - len(pt_tokens))
    en_padded = en_tokens[:MAX_LENGTH] + [0] * max(0, MAX_LENGTH - len(en_tokens))

    return tf.constant(pt_padded, dtype=tf.int64), tf.constant(en_padded, dtype=tf.int64)

# Apply the encoding and padding function
train_dataset = train_examples.map(lambda pt, en: tf.py_function(func=encode_and_pad, inp=[pt, en], Tout=[tf.int64, tf.int66]))
val_dataset = val_examples.map(lambda pt, en: tf.py_function(func=encode_and_pad, inp=[pt, en], Tout=[tf.int64, tf.int66]))

# Filter out empty sequences (which should not happen with start/end tokens, but as a safeguard)
train_dataset = train_dataset.filter(lambda pt, en: tf.logical_and(tf.size(pt) > 0, tf.size(en) > 0))
val_dataset = val_dataset.filter(lambda pt, en: tf.logical_and(tf.size(pt) > 0, tf.size(en) > 0))


# Cache, shuffle, and batch
BUFFER_SIZE = 20000
BATCH_SIZE = 64

train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


NameError: name 'train_examples' is not defined

I cant fix :/ IDK

In [ ]:
def encode_and_pad(pt, en):
    pt_tokens = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(pt.numpy()) + [tokenizer_pt.vocab_size + 1]
    en_tokens = [tokenizer_en.vocab_size] + tokenizer_en.encode(en.numpy()) + [tokenizer_en.vocab_size + 1]

    # Pad and truncate
    pt_padded = pt_tokens[:MAX_LENGTH] + [0] * max(0, MAX_LENGTH - len(pt_tokens))
    en_padded = en_tokens[:MAX_LENGTH] + [0] * max(0, MAX_LENGTH - len(en_tokens))

    return tf.constant(pt_padded, dtype=tf.int64), tf.constant(en_padded, dtype=tf.int64)

# Apply the encoding and padding function
train_dataset = train_examples.map(lambda pt, en: tf.py_function(func=encode_and_pad, inp=[pt, en], Tout=[tf.int64, tf.int64]))
val_dataset = val_examples.map(lambda pt, en: tf.py_function(func=encode_and_pad, inp=[pt, en], Tout=[tf.int64, tf.int64]))

# Filter out empty sequences (which should not happen with start/end tokens, but as a safeguard)
train_dataset = train_dataset.filter(lambda pt, en: tf.logical_and(tf.size(pt) > 0, tf.size(en) > 0))
val_dataset = val_dataset.filter(lambda pt, en: tf.logical_and(tf.size(pt) > 0, tf.size(en) > 0))


# Cache, shuffle, and batch
BUFFER_SIZE = 20000
BATCH_SIZE = 64

train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)



training setup defined the loss function, optimizer, and metrics for training.


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# Define the loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

# Define the optimizer
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# Define training metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

Training loop


In [ ]:
import tensorflow as tf
import numpy as np
import time
import tensorflow_datasets as tfds

# Assume necessary components (Transformer, tokenizer_pt, tokenizer_en, MAX_LENGTH, d_model) are defined

def translate_sentence(sentence, transformer, tokenizer_pt, tokenizer_en, MAX_LENGTH, d_model):
    """
    Translates a sentence using the trained Transformer model.
        sentence: The source sentence string (Portuguese).
        transformer: The trained Transformer model.
        tokenizer_pt: The source language (Portuguese) tokenizer.
        tokenizer_en: The target language (English) tokenizer.
        MAX_LENGTH: Maximum sequence length.
        d_model: Model dimensionality.

    Returns: translated_sentence: The translated sentence string (English).
    """
    # Encode the source sentence and prepare input
    sentence = tf.constant(sentence, dtype=tf.string)
    sentence = sentence.numpy().decode('utf-8')
    sentence = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(sentence) + [tokenizer_pt.vocab_size + 1]
    sentence = tf.keras.preprocessing.sequence.pad_sequences([sentence],
                                                             maxlen=MAX_LENGTH,
                                                             padding='post')
    encoder_input = tf.cast(sentence, dtype=tf.int64)

    # Create encoder padding mask
    enc_padding_mask = create_padding_mask(encoder_input)

    # 2. Obtain encoder output (evaluation mode)
    encoder_output = transformer.encoder(encoder_input, training=False, mask=enc_padding_mask)
    # encoder_output.shape == (1, MAX_LENGTH, d_model)

    # Initialize decoder input with the start token

    start_token = tokenizer_en.vocab_size
    end_token = tokenizer_en.vocab_size + 1
    decoder_input = tf.fill([1, MAX_LENGTH], 0, dtype=tf.int64)
    decoder_input = tf.tensor_scatter_nd_update(decoder_input, [[0, 0]], [start_token]) # Shape: (1, MAX_LENGTH)

    output_sequence = tf.TensorArray(dtype=tf.int64, size=MAX_LENGTH) # Store generated tokens

    # 4. Iterative decoding loop
    for i in range(MAX_LENGTH):
        current_decoder_input = decoder_input[:, :i+1] # Take up to current position
        # current_decoder_input.shape == (1, i+1)


        look_ahead_mask = create_look_ahead_mask(tf.shape(current_decoder_input)[1])

        dec_target_padding_mask = create_padding_mask(current_decoder_input)
        combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)


        predictions, attention_weights = transformer.decoder(
            current_decoder_input,
            encoder_output,
            training=False,
            look_ahead_mask=combined_mask,
            padding_mask=enc_padding_mask # Use encoder padding mask for cross-attention
        )


        # Pass decoder output through the final layer
        predictions = transformer.final_layer(predictions) # (1, i+1, target_vocab_size)


        predicted_token_id = tf.argmax(predictions[:, -1, :], axis=-1)[0] # Shape: scalar tf.int64

        # Store the predicted token
        output_sequence = output_sequence.write(i, predicted_token_id)

        # Check if the predicted token is the end token
        if predicted_token_id == end_token:
            break


        decoder_input = tf.tensor_scatter_nd_update(decoder_input, [[0, i + 1]], [predicted_token_id])


    # Decode the generated sequence of token IDs
    output_sequence = output_sequence.stack() # Shape: (generated_seq_len,)


    generated_tokens = output_sequence.numpy() # Convert to numpy array

    # Decode tokens back to string
    translated_sentence = tokenizer_en.decode([token for token in generated_tokens if token < tokenizer_en.vocab_size])

    return translated_sentence

BLEU score on the test set using the inference function.


Train the implemented Transformer model using the prepared dataset and training setup.


In [ ]:
import tensorflow as tf
import numpy as np # Keep numpy for positional_encoding function
import time # Keep time for timing

# Re-define the loss function and optimizer to ensure they are correctly initialized
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        # Explicitly cast step to float32
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)



learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')


# Define the training step with correct keyword arguments
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    # Create masks
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(inp=inp, tar=tar_inp,
                                     training=True,
                                     enc_padding_mask=enc_padding_mask,
                                     look_ahead_mask=combined_mask,
                                     dec_padding_mask=dec_padding_mask);
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss.update_state(loss)

    train_accuracy.update_state(tar_real, predictions)


# Main training loop
EPOCHS = 5

for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_state()
    train_accuracy.reset_state()

    for (batch, (inp, tar)) in enumerate(train_dataset):
        train_step(inp, tar)

        if batch % 50 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

    print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')
    print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')


Epoch 1 Batch 0 Loss 9.0133 Accuracy 0.0000
Epoch 1 Batch 50 Loss 8.9716 Accuracy 0.0003
Epoch 1 Batch 100 Loss 8.8838 Accuracy 0.0092
Epoch 1 Batch 150 Loss 8.7818 Accuracy 0.0167
Epoch 1 Batch 200 Loss 8.6548 Accuracy 0.0208
Epoch 1 Batch 250 Loss 8.5033 Accuracy 0.0232
Epoch 1 Batch 300 Loss 8.3305 Accuracy 0.0266
Epoch 1 Batch 350 Loss 8.1482 Accuracy 0.0304
Epoch 1 Batch 400 Loss 7.9743 Accuracy 0.0339
Epoch 1 Batch 450 Loss 7.8156 Accuracy 0.0372
Epoch 1 Batch 500 Loss 7.6721 Accuracy 0.0412
Epoch 1 Batch 550 Loss 7.5365 Accuracy 0.0454
Epoch 1 Batch 600 Loss 7.4082 Accuracy 0.0494
Epoch 1 Batch 650 Loss 7.2864 Accuracy 0.0534
Epoch 1 Batch 700 Loss 7.1683 Accuracy 0.0572
Epoch 1 Batch 750 Loss 7.0569 Accuracy 0.0611
Epoch 1 Batch 800 Loss 6.9514 Accuracy 0.0648
Epoch 1 Loss 6.9338 Accuracy 0.0654
Time taken for 1 epoch: 91.64 secs

Epoch 2 Batch 0 Loss 5.2590 Accuracy 0.1262
Epoch 2 Batch 50 Loss 5.2354 Accuracy 0.1275
Epoch 2 Batch 100 Loss 5.1964 Accuracy 0.1287
Epoch 2 Batch 

Implement the calculation of the BLEU score on the test set using the inference function.


In [ ]:
import nltk
import time

references = []
hypotheses = []


if 'test_examples' not in locals():
    print("Test dataset ('test_examples') not found. Using validation dataset ('val_examples') for BLEU evaluation.")
    test_examples = val_examples # Use validation data if test data is not loaded


num_test_examples = 100 # Evaluate on 100 examples

print(f"Evaluating BLEU score on {num_test_examples} examples...")
start_time = time.time()



for i, (pt_tensor, en_tensor) in enumerate(test_examples.take(num_test_examples)):
    try:
        # Get the translated sentence
        translated_sentence = translate_sentence(pt_tensor, transformer, tokenizer_pt, tokenizer_en, MAX_LENGTH, d_model)

        en_string = en_tensor.numpy().decode('utf-8')

        reference_words = en_string.split()
        hypothesis_words = translated_sentence.split()

        references.append([reference_words]) # List of lists for references
        hypotheses.append(hypothesis_words) # List of words for hypothesis

        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1}/{num_test_examples} examples.")

    except Exception as e:
        print(f"Error processing example {i + 1}: {e}")

        continue


try:
    bleu_score = nltk.translate.bleu_score.corpus_bleu(
        references,
        hypotheses,
        smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1
    )
    print(f"\nCorpus BLEU Score on {len(hypotheses)} examples: {bleu_score:.4f}")
except ZeroDivisionError:
    print("\nCould not compute BLEU score. This might happen if the generated hypotheses are empty.")
    bleu_score = 0.0
    print(f"Corpus BLEU Score: {bleu_score:.4f}")

end_time = time.time()
print(f"BLEU evaluation completed in {end_time - start_time:.2f} seconds.")

Evaluating BLEU score on 100 examples...
Processed 20/100 examples.
Error processing example 39: {{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:GPU:0}} indices[0] = [0, 40] does not index into shape [1,40] [Op:TensorScatterUpdate]
Processed 40/100 examples.
Processed 60/100 examples.


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3473, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)  File "/tmp/ipython-input-2517598569.py", line 32, in <cell line: 0>
    translated_sentence = translate_sentence(pt_tensor, transformer, tokenizer_pt, tokenizer_en, MAX_LENGTH, d_model)  File "/tmp/ipython-input-1264527923.py", line 75, in translate_sentence
    decoder_input = tf.tensor_scatter_nd_update(decoder_input, [[0, i + 1]], [predicted_token_id])  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/util/tf_shou

Processed 80/100 examples.
Error processing example 96: {{function_node __wrapped__TensorScatterUpdate_device_/job:localhost/replica:0/task:0/device:GPU:0}} indices[0] = [0, 40] does not index into shape [1,40] [Op:TensorScatterUpdate]
Processed 100/100 examples.

Corpus BLEU Score on 98 examples: 0.2033
BLEU evaluation completed in 108.22 seconds.


In [ ]:
import nltk
import time


references = []
hypotheses = []

if 'test_examples' not in locals():
    print("Test dataset ('test_examples') not found. Using validation dataset ('val_examples') for BLEU evaluation.")
    test_examples = val_examples # Use validation data if test data is not loaded


num_test_examples = 100 # Evaluate on 100 examples

print(f"Evaluating BLEU score on {num_test_examples} examples...")
start_time = time.time()



# Re-define the translate_sentence function to handle the MAX_LENGTH update
def translate_sentence(sentence, transformer, tokenizer_pt, tokenizer_en, MAX_LENGTH, d_model):
    """
    Translates a sentence using the trained Transformer model.

    Args:
        sentence: The source sentence tensor (Portuguese).
        transformer: The trained Transformer model.
        tokenizer_pt: The source language (Portuguese) tokenizer.
        tokenizer_en: The target language (English) tokenizer.
        MAX_LENGTH: Maximum sequence length.
        d_model: Model dimensionality.

    Returns:
        translated_sentence: The translated sentence string (English).
    """

    sentence = sentence.numpy().decode('utf-8')
    sentence = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(sentence) + [tokenizer_pt.vocab_size + 1]
    sentence = tf.keras.preprocessing.sequence.pad_sequences([sentence],
                                                             maxlen=MAX_LENGTH,
                                                             padding='post')
    encoder_input = tf.cast(sentence, dtype=tf.int64) # Shape: (1, MAX_LENGTH)

    # Create encoder padding mask
    enc_padding_mask = create_padding_mask(encoder_input)

    # Obtain encoder output
    encoder_output = transformer.encoder(encoder_input, training=False)

    # Initialize decoder input with the start token
    start_token = tokenizer_en.vocab_size
    end_token = tokenizer_en.vocab_size + 1
    decoder_input = tf.fill([1, MAX_LENGTH], tf.constant(0, dtype=tf.int64)) # Pass value and dtype together
    decoder_input = tf.tensor_scatter_nd_update(decoder_input, [[0, 0]], [start_token])

    output_sequence = tf.TensorArray(dtype=tf.int64, size=MAX_LENGTH)

    # decoding loop
    for i in range(MAX_LENGTH):
        current_decoder_input = decoder_input[:, :i+1]

        look_ahead_mask = create_look_ahead_mask(tf.shape(current_decoder_input)[1])
        dec_target_padding_mask = create_padding_mask(current_decoder_input)
        combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
        decoder_cross_attention_padding_mask = enc_padding_mask

        predictions, attention_weights = transformer.decoder(
            current_decoder_input,
            encoder_output,
            training=False,
            look_ahead_mask=combined_mask, # Mask for self-attention in decoder
            padding_mask=decoder_cross_attention_padding_mask # Mask for cross-attention in decoder
        )

        predictions = transformer.final_layer(predictions)

        predicted_token_id = tf.argmax(predictions[:, -1, :], axis=-1)[0]

        if i < MAX_LENGTH - 1:
             output_sequence = output_sequence.write(i, predicted_token_id)
        else:
             output_sequence = output_sequence.write(i, predicted_token_id)


        if tf.equal(predicted_token_id, end_token):
            break

        # Only update decoder_input if not at the last index of MAX_LENGTH
        if i < MAX_LENGTH - 1:
             decoder_input = tf.tensor_scatter_nd_update(decoder_input, [[0, i + 1]], [predicted_token_id])


    output_sequence = output_sequence.stack()


    generated_tokens = output_sequence.numpy()
    # Find the index of the first padding token (0) or end token
    try:
        first_padding_or_end_index = np.where((generated_tokens == 0) | (generated_tokens == end_token))[0][0]
        generated_tokens = generated_tokens[:first_padding_or_end_index + 1]
    except IndexError:

        pass



    translated_sentence = tokenizer_en.decode([token for token in generated_tokens if token < tokenizer_en.vocab_size])


    return translated_sentence


for i, (pt_tensor, en_tensor) in enumerate(test_examples.take(num_test_examples)):
    try:
        # Get the translated sentence
        translated_sentence = translate_sentence(pt_tensor, transformer, tokenizer_pt, tokenizer_en, MAX_LENGTH, d_model)

        en_string = en_tensor.numpy().decode('utf-8')

        reference_words = en_string.split()
        hypothesis_words = translated_sentence.split()

        references.append([reference_words]) # List of lists for references
        hypotheses.append(hypothesis_words) # List of words for hypothesis

        if (i + 1) % 20 == 0:
            print(f"Processed {i + 1}/{num_test_examples} examples.")

    except Exception as e:
        print(f"Error processing example {i + 1}: {e}")
        hypotheses.append([])

        try:
             pt_string_for_ref = pt_tensor.numpy().decode('utf-8')
             references.append([pt_string_for_ref.split()])
        except:
             references.append([["error"]])


# Calculate BLEU score

try:
    bleu_score = nltk.translate.bleu_score.corpus_bleu(
        references,
        hypotheses,
        smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1
    )
    print(f"\nCorpus BLEU Score on {len(hypotheses)} examples: {bleu_score:.4f}")
except ZeroDivisionError:
    print("\nCould not compute BLEU score. This might happen if the generated hypotheses are empty.")
    bleu_score = 0.0
    print(f"Corpus BLEU Score: {bleu_score:.4f}")

end_time = time.time()
print(f"BLEU evaluation completed in {end_time - start_time:.2f} seconds.")

Evaluating BLEU score on 100 examples...
Processed 20/100 examples.
Processed 40/100 examples.
Processed 60/100 examples.
Processed 80/100 examples.
Processed 100/100 examples.

Corpus BLEU Score on 100 examples: 0.1963
BLEU evaluation completed in 107.52 seconds.


BLEU Score Analysis: BLEU Score: 0.1963

About BLEU Score:
The model's generated translations have some overlap with the reference translations in terms of n-grams, but the overall quality is relatively low.

BLEU scores typically range from 0 to 1, where higher scores indicate better translation quality. So bad score.

Model Performance Discussion:
- This score suggests that the simplified Transformer model, trained on a subset of dataset for Portuguese to English translation, is possibly capable of generating some meaningful phrases and word sequences that align with the references, but it likely struggles with capturing the full meaning, fluency, and grammatical correctness of the target sentences.
- The relatively low score is because of the simplified architecture (2 encoder/decoder layers, scaled dot-product attention only) compared to full Transformer models, the limited training data size, and potentially limited training epochs (5 epochs).
